# WMT18

* To english
* Segment level data
* Relative ranking (better translation vs. worse translation)
* Kendall score

In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 4.7 MB/s 
     |████████████████████████████████| 101 kB 13.7 MB/s 
     |████████████████████████████████| 115 kB 67.4 MB/s 
     |████████████████████████████████| 141 kB 67.6 MB/s 
     |████████████████████████████████| 212 kB 68.4 MB/s 
     |████████████████████████████████| 596 kB 61.0 MB/s 
     |████████████████████████████████| 127 kB 77.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!git clone https://github.com/drehero/geneval

Cloning into 'geneval'...
remote: Enumerating objects: 286, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 286 (delta 27), reused 61 (delta 16), pack-reused 205
Receiving objects: 100% (286/286), 11.15 MiB | 9.08 MiB/s, done.
Resolving deltas: 100% (101/101), done.


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
import pathlib

import datasets
import numpy as np
import pandas as pd

from geneval.geneval.data.wmt import WMT18
from geneval.geneval.utils import kendall_score

In [ ]:
out_path = pathlib.Path("/content/drive/MyDrive/results/wmt18")
lang_pairs = ["cs-en", "de-en", "et-en", "fi-en", "ru-en", "tr-en", "zh-en"]

## BERT Score

In [ ]:
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 3.4 MB/s 
     |████████████████████████████████| 4.7 MB 9.4 MB/s 
     |████████████████████████████████| 6.6 MB 45.0 MB/s 


In [ ]:
metric_name = "bertscore"
metric_path = "bertscore"

target_lang = "en"
model_type = "bert-base-uncased"

In [ ]:
scorer = datasets.load_metric(metric_path)

In [ ]:
for lang_pair in lang_pairs:
    # load data
    wmt = WMT18(lang_pair, root="/tmp")

    sents = []
    refs = []
    for i, sent in enumerate(wmt.translations_better):
        if sent not in sents:
            sents += [sent]
            refs += [wmt.references[i]]
    for i, sent in enumerate(wmt.translations_worse):
        if sent not in sents:
            sents += [sent]
            refs += [wmt.references[i]]

    scores = scorer.compute(
        predictions=sents,
        references=refs,
        model_type=model_type,
        lang=target_lang,
    )

    sent2score = dict(zip(sents, scores["f1"]))

    # save scores
    scores_dict = {
        "translation_better": wmt.translations_better,
        "translations_worse": wmt.translations_worse,
        "reference": wmt.references,
        "source": wmt.sources,
        "score_better": [sent2score[sent] for sent in wmt.translations_better],
        "score_worse": [sent2score[sent] for sent in wmt.translations_worse]
    }
    df = pd.DataFrame(scores_dict)
    fn = f"{lang_pair}-{model_type}.csv"
    df.to_csv(out_path / metric_name / fn, index=False)

## BLEURT

In [ ]:
!pip install git+https://github.com/google-research/bleurt.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/google-research/bleurt.git to /tmp/pip-req-build-epqlnceo
  Running command git clone -q https://github.com/google-research/bleurt.git /tmp/pip-req-build-epqlnceo
     |████████████████████████████████| 352 kB 4.5 MB/s 
     |████████████████████████████████| 1.3 MB 8.9 MB/s 
  Created wheel for BLEURT: filename=BLEURT-0.0.2-py3-none-any.whl size=16456761 sha256=59315654661e2ae647215c9116a8f5de54e3d40c33a86bd2051aaa1111fcf5c0
  Stored in directory: /tmp/pip-ephem-wheel-cache-vvxv73lf/wheels/e2/2d/ea/b7a8b2424d2908d2a79d73ce8217d5ac4bd97ed3f47160a7f5
Successfully built BLEURT


In [ ]:
metric_name = "bleurt"
metric_path = "bleurt"

model_type = "bleurt-base-512"

In [ ]:
scorer = datasets.load_metric(metric_path, model_type)

In [ ]:
for lang_pair in lang_pairs:
    # load data
    wmt = WMT18(lang_pair, root="/tmp")

    sents = []
    refs = []
    for i, sent in enumerate(wmt.translations_better):
        if sent not in sents:
            sents += [sent]
            refs += [wmt.references[i]]
    for i, sent in enumerate(wmt.translations_worse):
        if sent not in sents:
            sents += [sent]
            refs += [wmt.references[i]]

    scores = scorer.compute(
        predictions=sents,
        references=refs
    )

    sent2score = dict(zip(sents, scores["scores"]))

    # save scores
    scores_dict = {
        "translation_better": wmt.translations_better,
        "translations_worse": wmt.translations_worse,
        "reference": wmt.references,
        "source": wmt.sources,
        "score_better": [sent2score[sent] for sent in wmt.translations_better],
        "score_worse": [sent2score[sent] for sent in wmt.translations_worse]
    }
    df = pd.DataFrame(scores_dict)
    fn = f"{lang_pair}-{model_type}.csv"
    df.to_csv(out_path / metric_name / fn, index=False)

## COMET

In [ ]:
!pip install unbabel-comet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 64 kB 2.6 MB/s 
     |████████████████████████████████| 409 kB 16.8 MB/s 
     |████████████████████████████████| 585 kB 57.3 MB/s 
     |████████████████████████████████| 1.2 MB 59.2 MB/s 
     |████████████████████████████████| 9.5 MB 50.7 MB/s 
     |████████████████████████████████| 116 kB 57.4 MB/s 
     |████████████████████████████████| 4.7 MB 47.6 MB/s 
     |████████████████████████████████| 101 kB 11.7 MB/s 
     |████████████████████████████████| 6.6 MB 51.3 MB/s 
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.5
    Uninstalling pandas-1.3.5:
      Successfully uninstalled pandas-1.3.5


In [ ]:
metric_name = "comet"
metric_path = "comet"

In [ ]:
scorer = datasets.load_metric(metric_path)

wmt20-comet-da.tar.gz: 1.79GB [00:44, 39.9MB/s]                            
Extracting /root/.cache/torch/unbabel_comet/wmt20-comet-da.tar.gz
Extracted /root/.cache/torch/unbabel_comet/wmt20-comet-da.tar.gz
Created a temporary directory at /tmp/tmp5oowzk58
Writing /tmp/tmp5oowzk58/_remote_module_non_scriptable.py


Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Encoder model frozen.


In [ ]:
for lang_pair in lang_pairs:
    # load data
    wmt = WMT18(lang_pair)

    sents = []
    refs = []
    srcs = []
    for i, sent in enumerate(wmt.translations_better):
        if sent not in sents:
            sents += [sent]
            refs += [wmt.references[i]]
            srcs += [wmt.sources[i]]
    for i, sent in enumerate(wmt.translations_worse):
        if sent not in sents:
            sents += [sent]
            refs += [wmt.references[i]]
            srcs += [wmt.sources[i]]

    scores = scorer.compute(
        predictions=sents,
        references=refs,
        sources=srcs,
        progress_bar=True
    )

    sent2score = dict(zip(sents, scores["scores"]))

    # save scores
    scores_dict = {
        "translation_better": wmt.translations_better,
        "translations_worse": wmt.translations_worse,
        "reference": wmt.references,
        "source": wmt.sources,
        "score_better": [sent2score[sent] for sent in wmt.translations_better],
        "score_worse": [sent2score[sent] for sent in wmt.translations_worse]
    }
    df = pd.DataFrame(scores_dict)
    fn = f"{lang_pair}.csv"
    df.to_csv(out_path / metric_name / fn, index=False)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 718/718 [02:33<00:00,  4.67it/s]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 3425/3425 [12:29<00:00,  4.57it/s]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0: 100%|██████████| 2521/2521 [09:02<00:00,  4.65it/s]
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Predicting DataLoader 0

## Frugalscore

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 15.0 MB/s 
     |████████████████████████████████| 6.6 MB 57.8 MB/s 


In [ ]:
metric_name = "frugalscore"
metric_path = "frugalscore"

model_type = "moussaKam/frugalscore_tiny_bert-base_bert-score"

max_length=512

batch_size=128
device="gpu"

In [ ]:
scorer = datasets.load_metric(metric_path, model_type)

In [ ]:
for lang_pair in lang_pairs:
    # load data
    wmt = WMT18(lang_pair)

    sents = []
    refs = []
    for i, sent in enumerate(wmt.translations_better):
        if sent not in sents:
            sents += [sent]
            refs += [wmt.references[i]]
    for i, sent in enumerate(wmt.translations_worse):
        if sent not in sents:
            sents += [sent]
            refs += [wmt.references[i]]

    scores = scorer.compute(
        predictions=sents,
        references=refs,
        max_length=max_length,
        batch_size=batch_size,
        device=device
    )

    sent2score = dict(zip(sents, scores["scores"]))

    # save scores
    scores_dict = {
        "translation_better": wmt.translations_better,
        "translations_worse": wmt.translations_worse,
        "reference": wmt.references,
        "source": wmt.sources,
        "score_better": [sent2score[sent] for sent in wmt.translations_better],
        "score_worse": [sent2score[sent] for sent in wmt.translations_worse]
    }
    df = pd.DataFrame(scores_dict)
    fn = f"{lang_pair}-{model_type.split('/')[-1]}.csv"
    df.to_csv(out_path / metric_name / fn, index=False)

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/28 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

  0%|          | 0/13 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/21 [00:00<?, ?ba/s]

## BARTScore

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 16.9 MB/s 
     |████████████████████████████████| 6.6 MB 63.6 MB/s 


In [ ]:
metric_name = "bartscore"
metric_path = "./geneval/geneval/metrics/bartscore/bartscore.py"

#model_type = "facebook/bart-base"
model_type = "facebook/bart-large-cnn"
max_length = 512
batch_size = 128

In [ ]:
scorer = datasets.load_metric(metric_path)

In [ ]:
for lang_pair in lang_pairs:
    # load data
    wmt = WMT18(lang_pair)

    sents = []
    srcs = []
    for i, sent in enumerate(wmt.translations_better):
        if sent not in sents:
            sents += [sent]
            srcs += [wmt.sources[i]]
    for i, sent in enumerate(wmt.translations_worse):
        if sent not in sents:
            sents += [sent]
            srcs += [wmt.sources[i]]

    scores = scorer.compute(
        predictions=sents,
        source_documents=srcs,
        model_type=model_type,
        max_length=max_length,
        batch_size=batch_size
    )

    sent2score = dict(zip(sents, scores))

    # save scores
    scores_dict = {
        "translation_better": wmt.translations_better,
        "translations_worse": wmt.translations_worse,
        "reference": wmt.references,
        "source": wmt.sources,
        "score_better": [sent2score[sent] for sent in wmt.translations_better],
        "score_worse": [sent2score[sent] for sent in wmt.translations_worse]
    }
    df = pd.DataFrame(scores_dict)
    fn = f"{lang_pair}-{model_type.split('/')[-1]}.csv"
    df.to_csv(out_path / metric_name / fn, index=False)

## MoverScore

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 14.1 MB/s 
     |████████████████████████████████| 6.6 MB 48.6 MB/s 


In [ ]:
metric_name = "moverscore"
metric_path = "geneval/geneval/metrics/moverScore/moverScore.py"

model_type = "bert-base-uncased"
batch_size = 128

In [ ]:
scorer = datasets.load_metric(metric_path)

In [ ]:
for lang_pair in lang_pairs:
    # load data
    wmt = WMT18(lang_pair)

    sents = []
    refs = []
    for i, sent in enumerate(wmt.translations_better):
        if sent not in sents:
            sents += [sent]
            refs += [wmt.references[i]]
    for i, sent in enumerate(wmt.translations_worse):
        if sent not in sents:
            sents += [sent]
            refs += [wmt.references[i]]

    scores = scorer.compute(
        predictions=sents,
        references=refs,
        model_type=model_type,
        n_gram=1,
        batch_size=batch_size
    )

    sent2score = dict(zip(sents, scores))

    # save scores
    scores_dict = {
        "translation_better": wmt.translations_better,
        "translations_worse": wmt.translations_worse,
        "reference": wmt.references,
        "source": wmt.sources,
        "score_better": [sent2score[sent] for sent in wmt.translations_better],
        "score_worse": [sent2score[sent] for sent in wmt.translations_worse]
    }
    df = pd.DataFrame(scores_dict)
    fn = f"{lang_pair}-{model_type.split('/')[-1]}.csv"
    df.to_csv(out_path / metric_name / fn, index=False)